In [9]:
import os
import numpy as np
import pandas as pd

from Bio.Seq import Seq
from Bio import SeqIO
#from Bio.Alphabet import Alphabet
from Bio.SeqRecord import SeqRecord

pd.options.mode.chained_assignment = None
import time

DIR = r'D:\Sequencing Files\Exp27A'

In [10]:
#to get later reads put n=16000001, m=16000001, i>32000004

list_sequences = []
list_quality = []
barcodes = []
    
with open(os.path.join(DIR, 'Exp27A.fastq'), 'r') as f:  

        n=1
        m=3

        for i, line in enumerate(f):
            if i ==n:
                list_sequences.append(line[5:-1])
                barcodes.append(line[:5])
                n+=4
            if i == m:
                list_quality.append(line[:-1])
                m+=4
            if i>100002:
                break
            
print(len(list_sequences))
print(len(list_quality))
print(len(barcodes))

df_read = pd.DataFrame(
    {'barcode':barcodes,
     'sequence_read': list_sequences,
     'quality_read': list_quality,
    })
df_read.head(20)
    

25001
25001
25001


,barcode,sequence_read,quality_read
0,TGACC,AAGTAGGGGATCCAGGTACTCGGGTACATCTAGAGCCGACCGCGAT...,BBABBGF=FGGGGCCEFGGGGGGGGGGGGGGGCGFF>FCGGGGGGG...
1,AGCAT,AAGTAGGGGATCCAGGTATTGGAGTGGTACTAGAGCCGACCGCGAT...,B@BBBGGGGGGGGGFGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,ACAGT,AAGTAGGGGATCCAGGTCTGGTACATGATCTAGAGCCGACCGCGAT...,CCCCCGGGGGGGGGGGGGGGGGFGGGGGGGGGGGGGGGGGGGGGGG...
3,CTGTA,CAACGTGGCGATCTGACGGATTCGATTTCTACTGCCTCTGGGGCCA...,BBABBGGGGDGGGGG<EGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
4,GAGTC,CAACGTGGCTGTCATCCGTGGGAGGTTCGTTCGGGTTATTCTTGTG...,CCCCCEGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
5,AACGT,AAGTAGGGGATCCAGGTACTCGGGTACATCTAGAGCCGACCGCGAT...,ABBBCGGGGGGGGGCF;DGGGFEFFF<BEFEB1=FGCFDGGGGGAG...
6,CAATG,AAGTAGGGGATCCAGGCGGTGCGCAGTATCTAGAGCCGACCGCGAT...,CCCCCGGGGGGGGGFGGGGGGGGGGGFGGGGGGGGGGGGGGGGGGG...
7,GACAT,CAACGTGGCGATAAGACGGATCCGGTTGCCTCTGGGGCCAGATCGG...,3AB0A1?>FDGFGGDCEF=::E>CBF/E:EEG>0>E>G1FBG<9>0...
8,CTTGA,AAGTAGGGGATCCAGGTCTGGTACATGATCTAGAGCCGACCGCGAT...,<A?00?FGGGDCC<EEEGGE<FGGBBFGGEGF1FEGG<FFFGGGGG...
9,TGGAC,AAGTAGGGGATCCAGGGAATGTATCATTTCTAGAGCCGACCGCGAT...,=<=<AB>@FC@1=//==FGEG<0CFGGGGG1EGB<FE11:=E/<CD...


In [11]:
import regex

def filter_5_primer(sequence):
    matches = regex.search('CAACGTGGC', sequence, regex.BESTMATCH)
    if matches ==None:
        return ('no match')
    else:
        index_end = matches.end()
        if sequence[index_end-1] == 'C':
            return (index_end)
        else:
            return ('no C')
        
def filter_3_primer(sequence):
    matches = regex.search('GCCTCTGGGGCC', sequence, regex.BESTMATCH)
    if matches ==None:
        return ('no match')
    else:
        index_start = matches.start()
        if sequence[index_start] == 'G':
            return (index_start)
        else:
            return ('no G')
        
        
df_read['start_peptide'] = df_read['sequence_read'].apply(filter_5_primer)
df_read['end_peptide'] = df_read['sequence_read'].apply(filter_3_primer)

df_read.head(20)

,barcode,sequence_read,quality_read,start_peptide,end_peptide
0,TGACC,AAGTAGGGGATCCAGGTACTCGGGTACATCTAGAGCCGACCGCGAT...,BBABBGF=FGGGGCCEFGGGGGGGGGGGGGGGCGFF>FCGGGGGGG...,no match,no match
1,AGCAT,AAGTAGGGGATCCAGGTATTGGAGTGGTACTAGAGCCGACCGCGAT...,B@BBBGGGGGGGGGFGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,no match,no match
2,ACAGT,AAGTAGGGGATCCAGGTCTGGTACATGATCTAGAGCCGACCGCGAT...,CCCCCGGGGGGGGGGGGGGGGGFGGGGGGGGGGGGGGGGGGGGGGG...,no match,no match
3,CTGTA,CAACGTGGCGATCTGACGGATTCGATTTCTACTGCCTCTGGGGCCA...,BBABBGGGGDGGGGG<EGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,9,33
4,GAGTC,CAACGTGGCTGTCATCCGTGGGAGGTTCGTTCGGGTTATTCTTGTG...,CCCCCEGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,9,no match
5,AACGT,AAGTAGGGGATCCAGGTACTCGGGTACATCTAGAGCCGACCGCGAT...,ABBBCGGGGGGGGGCF;DGGGFEFFF<BEFEB1=FGCFDGGGGGAG...,no match,no match
6,CAATG,AAGTAGGGGATCCAGGCGGTGCGCAGTATCTAGAGCCGACCGCGAT...,CCCCCGGGGGGGGGFGGGGGGGGGGGFGGGGGGGGGGGGGGGGGGG...,no match,no match
7,GACAT,CAACGTGGCGATAAGACGGATCCGGTTGCCTCTGGGGCCAGATCGG...,3AB0A1?>FDGFGGDCEF=::E>CBF/E:EEG>0>E>G1FBG<9>0...,9,27
8,CTTGA,AAGTAGGGGATCCAGGTCTGGTACATGATCTAGAGCCGACCGCGAT...,<A?00?FGGGDCC<EEEGGE<FGGBBFGGEGF1FEGG<FFFGGGGG...,no match,no match
9,TGGAC,AAGTAGGGGATCCAGGGAATGTATCATTTCTAGAGCCGACCGCGAT...,=<=<AB>@FC@1=//==FGEG<0CFGGGGG1EGB<FE11:=E/<CD...,no match,no match


In [12]:
with pd.ExcelWriter('exp27.xlsx') as writer:
    df_read.to_excel(writer, sheet_name='sequences_exp27')

In [1]:
# I added a test line
a = 4+4
print (a)

8


In [ ]:
# I added another test line
b = 8+8
print (b)